In [1]:
import pandas as pd
from sklearn.datasets import load_iris
 
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['class'] = iris.target
df['class name'] = iris.target_names[iris['target']]
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class,class name
0,5.1,3.5,1.4,0.2,0,setosa
1,4.9,3.0,1.4,0.2,0,setosa
2,4.7,3.2,1.3,0.2,0,setosa
3,4.6,3.1,1.5,0.2,0,setosa
4,5.0,3.6,1.4,0.2,0,setosa


In [2]:
from sklearn.model_selection import train_test_split
 
x_train, x_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=0)

In [3]:
from sklearn.neighbors import KNeighborsClassifier
 
model = KNeighborsClassifier()
model.fit(x_train, y_train)

KNeighborsClassifier()

In [4]:
model.score(x_test, y_test)

0.9666666666666667

In [5]:
model.predict([[5.6, 4.4, 1.2, 0.4]])

array([0])